In [1]:
%load_ext autoreload
%autoreload 1

In [3]:
%aimport utils

In [10]:
import numpy as np
import pandas as pd
import random
from utils import pairs2dict, run_experiment, AUC

In [5]:
df_item2artist = pd.read_csv('/mnt/workspace/Behance/Behance_Item_to_Owners', sep=" ",
                            header=None, names=['item', 'artist'])

In [6]:
item2artists = pairs2dict(df_item2artist.values)

In [7]:
def fa_score(artist_freqs, item):
    return sum(artist_freqs.get(a,0) for a in item2artists[item])

In [8]:
def compute_AUC(train_items, test_pos_items, test_neg_items):
    
    # compute artists frequencies in train items
    artist_freqs = dict()    
    for i in train_items:
        for a in item2artists[i]:
            artist_freqs[a] = artist_freqs.get(a, 0) + 1
    assert len(artist_freqs) > 0
    assert all(x > 0 for x in artist_freqs.values())    
    
    # rank test items
    pos_items_set = set(test_pos_items)
    test_items = test_pos_items + test_neg_items
    random.shuffle(test_items)
    test_items.sort(key=lambda i:fa_score(artist_freqs, i), reverse=True)
    
    # compute AUC
    relevant_positions = [pos for pos, i in enumerate(test_items) if i in pos_items_set]
    return AUC(relevant_positions, len(test_items))

In [11]:
run_experiment(compute_AUC, '/mnt/workspace/Behance/ExperimentResults/', 'pFA')

experiment successfully finished: results saved to /mnt/workspace/Behance/ExperimentResults/pFA_aucs.npy
	 elapsed_seconds = 5.32, mean_AUC = 0.65470
